# Burgers' equation

The inviscid Burgers' equation in one space dimension is
$$
\partial_t u + \partial_x \frac{u^2}{2} = 0.
$$
It is a scalar conservation law with convex flux $f(u) = \frac{u^2}{2}$.

## Riemann Problem

In [ ]:
using HyperbolicDiffEq
using Plots; pyplot()

# create a Riemann problem with left value 0 and right value 1
prob1 = RiemannProblem(Burgers(), 0., 1.)
sol1  = solve(prob1)
plot1 = plot(sol1)

# create a second Riemann problem at x₀=1
prob2 = RiemannProblem(Burgers(), 1., -1., 1.)
sol2  = solve(prob2)
plot2 = plot(sol2)

plot(plot1, plot2)

### Consecutive Riemann Problems

In [ ]:
prob3 = RiemannProblem(Burgers(), -1., -2., 2)

prob = prob1 * prob2
prob = prob * prob3
sol = solve(prob)
plot(sol)